# Imports

In [ ]:
import pandas as pd
import numpy as np

# 1️⃣ Pipelines

- Encoding kategorialer Merkmale (z. B. position, foot, country_of_citizenship)
- Feature-Skalierung = z. B. mit StandardScaler oder MinMaxScaler
- Dimensionsreduktion (optional) = z. B. PCA (from sklearn.decomposition import PCA)	
- from sklearn.model_selection import train_test_split = z. B. X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
- Modellauswahl / Training = z. B. LinearRegression, RandomForestRegressor, XGBoost, etc.

# 2️⃣ Hyperparameter-Tuning
- GridSearchCV, RandomSearch, Optuna, etc.

# 3️⃣ Evaluation
- Metriken: MAE, RMSE, R²
- Visualisierung: Predicted vs. Actual, Residualplots